# 목적
* 이 예제에서는 5개의 테이블에 대한 table description excel 파일을 Milvus DB에 넣는 실습을 진행합니다.
* 이 예제에서 사용하는 데이터는 `./data/`폴더 내에 있는 5개 엑셀 파일을 가지고 실습합니다.

In [1]:
import os
import json
from typing import Dict, List
import textwrap
import logging
import pandas as pd
from pathlib import Path, PosixPath
from dotenv import load_dotenv

from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_milvus import Milvus

from helper import find_project_root, load_tables_description


## 0. 프로젝트 경로 초기 셋팅

In [19]:
ROOT_PATH = find_project_root()
DATA_PATH = ROOT_PATH.joinpath("shilla/example_human_gen/data")

load_dotenv(ROOT_PATH.joinpath(".env"))


True

## 1. Table description 불러오기
* 미리 저장되어 있는 table description을 불러옵니다.
* 저장 경로는 DATA_PATH 입니다.

In [20]:
table_description = load_tables_description(DATA_PATH, False) # CHESS 함수 c/p


In [21]:
table_description


{'daily_finished_goods_agg': {'critn_date': {'original_column_name': 'critn_date',
   'column_name': 'creation date',
   'column_description': 'created year, month and date explained as 8 yyyymmdd digits',
   'data_format': 'text',
   'value_description': ''},
  'fact_nm': {'original_column_name': 'fact_nm',
   'column_name': 'factory name',
   'column_description': 'name of factory',
   'data_format': 'text',
   'value_description': ''},
  'line_cd': {'original_column_name': 'line_cd',
   'column_name': 'line code',
   'column_description': 'grouped process line code',
   'data_format': 'text',
   'value_description': ''},
  'line_nm': {'original_column_name': 'line_nm',
   'column_name': 'line name',
   'column_description': 'grouped process line name',
   'data_format': 'text',
   'value_description': ''},
  'item_cd': {'original_column_name': 'item_cd',
   'column_name': 'item code',
   'column_description': 'item code name',
   'data_format': 'text',
   'value_description': ''},
 

## 2. Document 형태로 재구성
* CHESS에 있는 코드를 참고하여 재구성
* LangChain에서 제공하는 클래스인 `Document`에 정의
  * page_content (유사도로 비교하는 실제 탐색 정보) -> table_name, column_name, column_description 추가
  * metadata (해당 정보에 대한 메타데이터) -> 나머지 정보 입력 (아래 코드 참고)
* page_content에 들어가는 정보가 핵심입니다. 사용자의 질문과 유사도를 계산하여 가장 유사도가 높은 정보를 찾은 실질적인 탐색정보.
* page_content에 되도록 많은 정보가 들어갈수록 성능은 무조건 좋아지지만, 실제 제품으로 사용할 때 모든 정보가 들어가지 않음.
* 데이터 거버넌스 측면에서 중 컬럼에 뜻, 메타 정보 등은 매우 중요하며, 미리 구축해두지 않았더라도, Vivity One을 사용하기 위해선 초기에 반드시 필요한 셋팅임.
* 따라서 column_description을 page_content에 배치시킴. 추가로 테이블 이름과 컬럼명을 같이 넣어 해당 column_description에 해당하는 테이블 명과 컬럼명을 같이 알 수 있게함.
* json 형태로 page_content에 추가함.

In [22]:
# table name docs
# CHESS 코드를 약간 변형함
def gen_docs(table_description: Dict[str, Dict])-> List[Document]:
    docs = []
    for table_name, columns in table_description.items():
        for column_name, column_info in columns.items():
            metadata = {
                "table_name": table_name,
                "original_column_name": column_name,
                "column_name": column_info.get('column_name', ''),
                "column_description": column_info.get('column_description', ''),
                "value_description": column_info.get('value_description', '') if True else ""
            }
            # print(column_info)
            # print("a")
            for key in ['column_description']:
                if column_info.get(key, '').strip():
                    json_str = {
                        "table": table_name,
                        "column": column_name,
                        "column_description": column_info.get(key, '')
                    }
                    page_content = textwrap.dedent(f'''
                    ```json
                    {json_str}
                    ```
                    ''')
                    docs.append(Document(page_content=page_content, metadata=metadata))
    return docs


In [23]:
docs = gen_docs(table_description)
len(docs)


63

In [24]:
docs[0:3]


[Document(metadata={'table_name': 'daily_finished_goods_agg', 'original_column_name': 'critn_date', 'column_name': 'creation date', 'column_description': 'created year, month and date explained as 8 yyyymmdd digits', 'value_description': ''}, page_content="\n```json\n{'table': 'daily_finished_goods_agg', 'column': 'critn_date', 'column_description': 'created year, month and date explained as 8 yyyymmdd digits'}\n```\n"),
 Document(metadata={'table_name': 'daily_finished_goods_agg', 'original_column_name': 'fact_nm', 'column_name': 'factory name', 'column_description': 'name of factory', 'value_description': ''}, page_content="\n```json\n{'table': 'daily_finished_goods_agg', 'column': 'fact_nm', 'column_description': 'name of factory'}\n```\n"),
 Document(metadata={'table_name': 'daily_finished_goods_agg', 'original_column_name': 'line_cd', 'column_name': 'line code', 'column_description': 'grouped process line code', 'value_description': ''}, page_content="\n```json\n{'table': 'daily_f

## 3. Milvus vector store 저장
* 임베딩 함수는 openai 임베딩 함수를 사용함.
* 이는 추후 로컬 모델로 변경해서 테스트 해야함.
* 본 예제에서는 AI01에 데이터를 넣는 예제임.
  * (한번 생성한 collenction이 있는데 from_document로 넣으면 그냥 넘기는 듯 합니다)

In [25]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


In [26]:
# AI01 Standalone Milvus DB
URI = "http://10.128.0.20:19530"


In [27]:
vector_store_saved = Milvus.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name="shilla_data_descriptions",
    connection_args={"uri":URI}
)


## 4. Milvus vector store 데이터 조회
* Milvus DB에서 데이터 조회, 컬렉션 리스트 조회 등 가능.

In [28]:
from pymilvus import Collection, utility, connections

# 1. Milvus 서버에 접속 (URI 입력)
connections.connect(alias = "standalone-ai01", uri=URI)

# 2. 전체 컬렉션 리스트
collection_names = utility.list_collections(using="standalone-ai01")

# 3. 컬렉션 객체 불러오기
collection = Collection("shilla_data_descriptions", using="standalone-ai01") 

collection_names, collection


(['shilla_test_20250526_openai_embedding_3_large',
  'shilla_data_descriptions',
  'shilla_test_20250526_ollama_bge_m3_1',
  'shilla_test_20250526_ollama_bge_m3',
  'shilla_test_20250526_openai_embedding_3_large_1'],
 <Collection>:
 -------------
 <name>: shilla_data_descriptions
 <description>: 
 <schema>: {'auto_id': True, 'description': '', 'fields': [{'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 3072}}, {'name': 'table_name', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'original_column_name', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'column_name', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'n

In [29]:
for field in collection.schema.fields:
    print(f" - 이름: {field.name}, 타입: {field.dtype}, 벡터 차원: {getattr(field, 'dim', 'N/A')}")


 - 이름: text, 타입: 21, 벡터 차원: None
 - 이름: pk, 타입: 5, 벡터 차원: None
 - 이름: vector, 타입: 101, 벡터 차원: 3072
 - 이름: table_name, 타입: 21, 벡터 차원: None
 - 이름: original_column_name, 타입: 21, 벡터 차원: None
 - 이름: column_name, 타입: 21, 벡터 차원: None
 - 이름: column_description, 타입: 21, 벡터 차원: None
 - 이름: value_description, 타입: 21, 벡터 차원: None


## 5. Milvus vector store 검색
* 유사도 검색하는 방법
* 유사도 점수까지 함깨 내는 방법 
* retriever로 사용하는 방법
  * filter 같은 파라미터를 사용하면 metadata에서 먼저 filtering 거친 후 탐색할 수도 있음 (탐색범위를 줄여주는)

In [30]:
vector_store_loaded = Milvus(
    embedding_function=embeddings,
    collection_name="shilla_data_descriptions",
    connection_args={"uri": URI}
)


In [31]:
results = vector_store_loaded.similarity_search(
    "What is the average work time for the 정삭 process at the 장산공장 plant, produced in May 2024?",
    k=3
)
results


[Document(metadata={'table_name': 'monthly_production_metrics', 'original_column_name': 'work_time', 'column_name': 'work time', 'column_description': 'period of time when the plant is operated', 'value_description': '', 'pk': 458330752954111767}, page_content="\n```json\n{'table': 'monthly_production_metrics', 'column': 'work_time', 'column_description': 'period of time when the plant is operated'}\n```\n"),
 Document(metadata={'table_name': 'monthly_production_metrics', 'original_column_name': 'work_time', 'column_name': 'work time', 'column_description': 'period of time when the plant is operated', 'value_description': '', 'pk': 458330752954111839}, page_content="\n```json\n{'table': 'monthly_production_metrics', 'column': 'work_time', 'column_description': 'period of time when the plant is operated'}\n```\n"),
 Document(metadata={'table_name': 'monthly_production_metrics', 'original_column_name': 'work_time', 'column_name': 'work time', 'column_description': 'period of time when th

In [32]:
results = vector_store_loaded.similarity_search_with_score(
    "What is the average work time for the 정삭 process at the 장산공장 plant, produced in May 2024?",
    k=3
)
results


[(Document(metadata={'table_name': 'monthly_production_metrics', 'original_column_name': 'work_time', 'column_name': 'work time', 'column_description': 'period of time when the plant is operated', 'value_description': '', 'pk': 458330752954111767}, page_content="\n```json\n{'table': 'monthly_production_metrics', 'column': 'work_time', 'column_description': 'period of time when the plant is operated'}\n```\n"),
  1.3856805562973022),
 (Document(metadata={'table_name': 'monthly_production_metrics', 'original_column_name': 'work_time', 'column_name': 'work time', 'column_description': 'period of time when the plant is operated', 'value_description': '', 'pk': 458330752954111839}, page_content="\n```json\n{'table': 'monthly_production_metrics', 'column': 'work_time', 'column_description': 'period of time when the plant is operated'}\n```\n"),
  1.3856805562973022),
 (Document(metadata={'table_name': 'monthly_production_metrics', 'original_column_name': 'work_time', 'column_name': 'work tim

In [33]:
retriever = vector_store_loaded.as_retriever(search_type="mmr", search_kwargs={"k": 3})                 
retriever.invoke("What is the average work time for the 정삭 process at the 장산공장 plant, produced in May 2024?")


[Document(metadata={'table_name': 'monthly_production_metrics', 'original_column_name': 'work_time', 'column_name': 'work time', 'column_description': 'period of time when the plant is operated', 'value_description': '', 'pk': 458330752954111767}, page_content="\n```json\n{'table': 'monthly_production_metrics', 'column': 'work_time', 'column_description': 'period of time when the plant is operated'}\n```\n"),
 Document(metadata={'table_name': 'monthly_production_metrics', 'original_column_name': 'mttr', 'column_name': 'Mean Time To Repair', 'column_description': 'mean time to repair which represents maintenance of equipment. Calculated with this formula, round(breakdown_time / nullif(breakdown_cnt, 0), 0)', 'value_description': '', 'pk': 458330752954111782}, page_content="\n```json\n{'table': 'monthly_production_metrics', 'column': 'mttr', 'column_description': 'mean time to repair which represents maintenance of equipment. Calculated with this formula, round(breakdown_time / nullif(br